# Mom Wellness Agent

This notebook generates a personalized workout plan for a 60-year-old healthy woman.

In [99]:
from dotenv import load_dotenv
import os
load_dotenv(override=True)
from openai import OpenAI
from IPython.display import display, Markdown
from anthropic import Anthropic

import importlib
import db
importlib.reload(db)
from db import init_db, save_plan, get_plans

openai_api_key = os.getenv('OPENAI_API_KEY')
openai = OpenAI()
DEFAULT_PROFILE = "60yo_woman"
init_db()

In [96]:
from pydantic import BaseModel
class Evaluation(BaseModel):
    is_acceptable:bool
    feedback:str




In [21]:
evaluator_system_prompt = "you are are an evaluator that decides whether a response to a question is acceptable\
     your job is to evaluate whether the agent is playing the role of a wellness coach and designing\
        weekly workout plans that are based on sound science, have not gym depedencies, are suitable for \
            women of 60 years old. the plans should be practical and effective in regulating weight, healthy cardiovascular\
                healthy weight loss "

In [22]:
def evaluator_user_prompt(reply, message, history):
    user_prompt = f"Here's the conversation between the User and the Agent: \n\n{history}\n\n"
    user_prompt += f"Here's the latest message from the User: \n\n{message}\n\n"
    user_prompt += f"Here's the latest response from the Agent: \n\n{reply}\n\n"
    user_prompt += "Please evaluate the response, replying with whether it is acceptable and your feedback."
    return user_prompt

In [23]:
import os
claude = Anthropic()




In [24]:
def Evaluator(reply, message, history) -> Evaluation:
    # Use messages.parse() with output_format=YourBaseModel to get structured output.
    # System prompt is passed via system=, not as a message.
    response = claude.messages.parse(
        model="claude-sonnet-4-5",
        system=evaluator_system_prompt,
        messages=[{"role": "user", "content": evaluator_user_prompt(reply, message, history)}],
        max_tokens=1000,
        output_format=Evaluation,
    )
    # parsed_output is the Pydantic instance when using output_format
    return response.parsed_output



In [47]:
from pydantic import BaseModel
class responseModelFormat(BaseModel):
    monday:str
    monday_workout:str
    tuesday:str
    tuesday_workout:str
    wednesday:str
    wednesday_workout:str
    thursday:str
    thursday_workout:str
    friday:str
    friday_workout:str
    saturday:str
    saturday_workout:str
    sunday:str
    sunday_workout:str


In [71]:
agent_wellness_prompt = "design a week long workout plan for a 60 year old healthy woman. No gym equipment. walking and body exercises only.\
     weight loss is a goal and overall fitness. tuesday and thursdays should be easy mode workouts"
messages = [{"role":"system", "content": agent_wellness_prompt}] + [{"role":"user", "content": agent_wellness_prompt}]
responses = openai.chat.completions.parse(
    model="gpt-4o-mini",
    messages=messages,
    response_format=responseModelFormat
)
answer = responses.choices[0].message.parsed

In [82]:
display(answer)
display(answer.monday_workout)

responseModelFormat(monday='Monday', monday_workout='1. Warm-up: 5-10 minutes brisk walking. \n2. Bodyweight Squats: 3 sets of 10-15 reps. \n3. Incline Push-ups (on a wall or table): 3 sets of 8-10 reps. \n4. Standing Calf Raises: 3 sets of 15 reps. \n5. Step-ups (on steps): 3 sets of 10 reps each leg. \n6. Cool down: 5-10 minutes walking followed by stretching.', tuesday='Tuesday', tuesday_workout='1. Warm-up: 5 minutes slow walking.\n2. Gentle Yoga or Stretching Routine: 15-20 minutes (include neck, back, shoulders, and legs).\n3. Easy Walk: 20-30 minutes at a comfortable pace.\n4. Cool down: gentle stretching for 5-10 minutes.', wednesday='Wednesday', wednesday_workout='1. Warm-up: 5-10 minutes brisk walking. \n2. Walking Lunges: 3 sets of 10 reps each leg. \n3. Tricep Dips (using a chair): 3 sets of 8-10 reps. \n4. Side Leg Raises: 3 sets of 10-15 reps each leg. \n5. Beginner Plank (on knees if needed): Hold for 15-30 seconds, 3 times. \n6. Cool down: 5-10 minutes walking followed 

'1. Warm-up: 5-10 minutes brisk walking. \n2. Bodyweight Squats: 3 sets of 10-15 reps. \n3. Incline Push-ups (on a wall or table): 3 sets of 8-10 reps. \n4. Standing Calf Raises: 3 sets of 15 reps. \n5. Step-ups (on steps): 3 sets of 10 reps each leg. \n6. Cool down: 5-10 minutes walking followed by stretching.'

In [76]:
evaluation_response = Evaluator(answer, agent_wellness_prompt, messages[:1])

In [77]:
evaluation_response

Evaluation(is_acceptable=True, feedback='This is an excellent, well-designed weekly workout plan that demonstrates strong coaching in wellness for a 60-year-old woman. The plan successfully addresses all requirements: it uses only walking and bodyweight exercises with no gym equipment, appropriately includes easier workouts on Tuesdays and Thursdays as requested, and focuses on weight loss and overall fitness goals. The program shows sound exercise science principles with proper warm-ups, cool-downs, stretching, and progressive intensity. The variety includes cardiovascular work (walking), strength training (squats, lunges, step-ups, push-ups), core work (planks, bicycle crunches), flexibility (yoga, stretching), and balance components. The modifications offered (like incline push-ups on walls, planks on knees) are age-appropriate and safe. The plan appropriately balances harder workout days (Monday, Wednesday, Friday) with easier recovery days (Tuesday, Thursday) and includes active r

In [83]:
def rerun(reply, message, history, feedback) -> responseModelFormat:
    updated_system_prompt = agent_wellness_prompt + "\n\n## Previous answer rejected\nYou just tried to reply, but the quality control rejected your reply\n"
    updated_system_prompt += f"## Your attempted answer:\n{reply}\n\n"
    updated_system_prompt += f"## Reason for rejection:\n{feedback}\n\n"
    messages = [{"role": "system", "content": updated_system_prompt}] + history + [{"role": "user", "content": message}]
    response = openai.chat.completions.parse(model="gpt-4o-mini", messages=messages, response_format=responseModelFormat)
    return response.choices[0].message.parsed

In [101]:
if not evaluation_response.is_acceptable:
    retry_message = [{"role": "system", "content": agent_wellness_prompt}] + [{"role": "user", "content": agent_wellness_prompt}]
    history = []  # no prior conversation on first run
    second_attempt = rerun(answer, agent_wellness_prompt, history, evaluation_response.feedback) 
    final_answer = second_attempt
    save_plan("mymother", second_attempt.monday_workout, second_attempt.monday)
    save_plan("mymother", second_attempt.tuesday_workout, second_attempt.tuesday)
    save_plan("mymother", second_attempt.wednesday_workout, second_attempt.wednesday)
    save_plan("mymother", second_attempt.thursday_workout, second_attempt.thursday)
    save_plan("mymother", second_attempt.friday_workout, second_attempt.friday)
    save_plan("mymother", second_attempt.saturday_workout, second_attempt.saturday)
    save_plan("mymother", second_attempt.sunday_workout, second_attempt.sunday)
else:
    final_answer = answer
    save_plan("mymother", answer.monday_workout, answer.monday)
    save_plan("mymother", answer.tuesday_workout, answer.tuesday)
    save_plan("mymother", answer.wednesday_workout, answer.wednesday)
    save_plan("mymother", answer.thursday_workout, answer.thursday)
    save_plan("mymother", answer.friday_workout, answer.friday)
    save_plan("mymother", answer.saturday_workout, answer.saturday)
    save_plan("mymother", answer.sunday_workout, answer.sunday)  



In [103]:
print(get_plans("mymother"))

1. Warm-up: 5-10 minutes brisk walking. 
2. Walking Lunges: 3 sets of 10 reps each leg. 
3. Tricep Dips (using a chair): 3 sets of 8-10 reps. 
4. Side Leg Raises: 3 sets of 10-15 reps each leg. 
5. Beginner Plank (on knees if needed): Hold for 15-30 seconds, 3 times. 
6. Cool down: 5-10 minutes walking followed by stretching.
